# DS861 Homework 2

In this assignment, we will be analyzing crime rate data from different communities. We will ran three different Lasso regression models (train/validation/test split, 5-fold cross validation and 10-fold cross validation) and compare their performances. 

## Running 3 Methods

In [1]:
# first, let's import some libraries we need to use in this exercise.
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
#basic libraries we use everytime

from sklearn import linear_model # For LASSO 
from sklearn.linear_model import Lasso # For LASSO
from sklearn.metrics import mean_squared_error # For evaluation
from sklearn.preprocessing import StandardScaler #For standardlization
from sklearn.model_selection import train_test_split #For data split
from sklearn import metrics # For evaluation

from sklearn.model_selection import GridSearchCV #For hyperparameters
from sklearn.pipeline import Pipeline
import itertools 

import warnings # Suppress warnings because they are annoying
warnings.filterwarnings('ignore') 

import time #Help you deal with running time

The original data set can be found on the UCI machine learning data repository  https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized). 

The final data set that we will use (community.csv) consists of 2118 observations, and 101 predictors + 1 response (total number of non-violent crimes).

Since the data has been clean up, there is no missing value or unnessary response variables, we are ready to use it for our models.

In [2]:
# Load the data
community = pd.read_csv("community.csv")
community.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,nonViolPerPop
0,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,11.33,...,10.66,53.72,65.29,78.09,89.14,6.5,1845.9,9.63,0.0,1394.59
1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,17.18,...,8.30,77.17,71.27,90.22,96.12,10.6,2186.7,3.84,0.0,1955.95
2,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,10.28,...,5.00,44.77,36.60,61.26,82.85,10.6,2780.9,4.37,0.0,6167.51
3,11245,2.76,0.53,89.16,1.17,0.52,24.46,40.53,28.69,12.65,...,1.74,73.75,42.22,60.34,89.02,11.5,974.2,0.38,0.0,9988.79
4,140494,2.45,2.51,95.65,0.90,0.95,18.09,32.89,20.04,13.26,...,1.49,64.35,42.29,70.61,85.66,70.4,1995.7,0.97,0.0,6867.42


In [3]:
# we want to see all the variable names.
NameOfVariables = community.columns.values
NameOfVariables

array(['population', 'householdsize', 'racepctblack', 'racePctWhite',
       'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29',
       'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome',
       'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec',
       'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc',
       'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap',
       'HispPerCap', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade',
       'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy',
       'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu',
       'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr',
       'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par',
       'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par',
       'PctWorkMomYoungKids', 'PctWorkMom', 'NumKidsBornNeverMar',
       'PctKidsBornNeverMar', 'NumImmig', 'PctImmigRecent',
       'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig',
       'Pc

From the outout, we can see all the variables that are been used in this dataset. The detailed explination of them can be seen from the website link.

The response variable we use in the study is nonViolPerPop, which shows total number of non-violent crimes per 100K popuation (numeric - decimal) potential GOAL attribut.

In [4]:
# Separate the X and Y
# nonViolPerPop is our response
X = community.copy() 
del X['nonViolPerPop']
y = community['nonViolPerPop']

We would like to have 30% as our hold-out set, and use 70% as training.

In [5]:
# use 30% as your hold-out set)
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, 
                                                                test_size = 0.3, random_state = 1)

### First method: Train / validation / test split.

Normally, we will choose 60%-20%-20% to split the data. Since we want to have 30% hold-out, we use 56%-14%-30% split for this model.

In [6]:
# Now split the training set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, 
                                                      test_size = 0.2, random_state = 1) 
# For the entire dataset, we will do a 56%-14%-30% split.

First, we need to standardlize the data, we fit the data with training set, then apply this to training and validation sets.

In [7]:
# Scale data (only fit with the training data, and transform both the training and validation data)
scaler = StandardScaler() # Instantiate
scaler.fit(X_train) # Fit the data
X_train = pd.DataFrame(scaler.transform(X_train)) # Transform the data
X_valid = pd.DataFrame(scaler.transform(X_valid)) # Transform the validation set
X_train.columns = X.columns.values
X_valid.columns = X.columns.values

The hyperparameters for the models includes: alpha (the lambda value in the penalty), max_iter (the maximum number of iterations for optimization algorithm) and tol (the tolerance for optimization).

Let's define the hyperparameters we will use for the models. Total, we have 630 pairs.

In [8]:
# Predefine the hyperparameters
lambdas = np.logspace(-10,10,21)   # We will use lambda on powers of 10 scale
max_iter = [50,70,90]   #We select three numbers
tol = np.linspace(0,0.1,10)  #We select 10 numbers from 0 to 0.1
hyperparameter_pairs = list(itertools.product(lambdas, max_iter, tol))
len(hyperparameter_pairs) 

630

In [9]:
start = time.time()

Validation_Scores = []     
for pairs in hyperparameter_pairs:
    lm = Lasso(alpha = pairs[0], max_iter = pairs[1], tol = pairs[2])
    lm.fit(X_train, y_train) # Fit model on training set
    Validation_Scores.append(metrics.mean_squared_error(lm.predict(X_valid), y_valid)) 
    # Evaluate model on validation set

# Find the minimum validation error, and it's minimizer
min_mse = min(Validation_Scores)
best_pairs = hyperparameter_pairs[np.argmin(Validation_Scores)]

print("Minimum MSE on valid set: ", min_mse)    
print("Best_pairs:", best_pairs)

end = time.time()
print("Total time we use is: "+str(end-start))

Minimum MSE on valid set:  3179290.9583979663
Best_pairs: (1.0, 50, 0.0)
Total time we use is: 3.8457589149475098


When we make a Lasso Regression through Train/validation/test split model, we notice that the total time is around 3.85 seconds, which is quite fast. We want to find the best parameters with the lowest mean squared error. In this case, the alpha is 1, max_iter is 50 and tol is 0.

In [10]:
# Scale data (use the train_valid set to fit, then transform both the train_valid set and test set)
scaler = StandardScaler() # Instantiate
scaler.fit(X_train_valid) # First fit the data, i.e. learn the mean and sd

X_train_valid = pd.DataFrame(scaler.transform(X_train_valid)) # Then transform the data. We can also use fit_transform
X_test = pd.DataFrame(scaler.transform(X_test))

# Remember, the steps are fit, then transform

# Refit Lasso model with selected alpha value (10 from the lambda we get before)
lm1 = Lasso(alpha = best_pairs[0], max_iter = best_pairs[1], tol = best_pairs[2])
lm1.fit(X_train_valid, y_train_valid)

#sort the variables
Var_coef = zip(lm1.coef_ , NameOfVariables)
sorted(Var_coef)

[(-744.2533068136923, 'NumKidsBornNeverMar'),
 (-669.8374700503999, 'PersPerOwnOccHous'),
 (-464.40249716283387, 'PctImmigRec5'),
 (-456.8441432163033, 'PctRecentImmig'),
 (-447.7657445112417, 'PctFam2Par'),
 (-424.6782049090016, 'PctKids2Par'),
 (-385.46822694799795, 'PersPerRentOccHous'),
 (-375.7178109821489, 'PctBSorMore'),
 (-368.4473411507553, 'PctLess9thGrade'),
 (-334.88499697124695, 'PctPersOwnOccup'),
 (-330.61631249825325, 'OwnOccLowQuart'),
 (-292.08736958795254, 'PopDens'),
 (-287.0065360886346, 'PctSpeakEnglOnly'),
 (-262.53180829090746, 'householdsize'),
 (-256.0140579872967, 'PctLargHouseFam'),
 (-238.23684429507063, 'PctPersDenseHous'),
 (-236.78129402293172, 'PctHousLess3BR'),
 (-234.47649553348887, 'MedOwnCostPctInc'),
 (-228.75992451973983, 'PctVacMore6Mos'),
 (-224.15623327125536, 'RentLowQ'),
 (-198.9808571207515, 'agePct12t29'),
 (-192.73504529415507, 'medFamInc'),
 (-191.3077149088405, 'population'),
 (-188.89679856907168, 'PctEmplManu'),
 (-182.5233471716151, '

In [11]:
#coefficients shriked to 0 with Train / validation / test split method
lm1_coef=pd.DataFrame(zip(lm1.coef_, X_train_valid.columns.values))
len(lm1_coef.loc[lm1_coef[0] == 0])

1

In [12]:
print("The prediction error on the testing set is", metrics.mean_squared_error(lm1.predict(X_test), y_test))

The prediction error on the testing set is 3539035.5943705076


Then, we fit the data with training plus validation set, and transform it to testing set. The result shows a prediction error on the testing set is 3539035.59, which we will compare it with the results of method 2 and 3. The coefficient of PctEmplProfServ is shrink to zero.

### Second method: 5-Fold cross validation.

In [13]:
start = time.time()

best_param=[] # Store the best hyperparameter results

# Set up the model pipeline
estimator = Pipeline(steps = [('scale', StandardScaler()), # We first need to scale the data
                     ('lasso', Lasso()) ]) # Then fit the scaled data using Lasso

# Set up the parameters for each item in the pipeline
# Parameters of pipelines can be set using ‘__’ separating the parameter names:
parameters = {'lasso__alpha': lambdas,
             'lasso__max_iter': max_iter,
             'lasso__tol': tol} 

reg = GridSearchCV(estimator = estimator, param_grid = parameters, cv = 5, 
                   scoring = 'neg_mean_squared_error', n_jobs = -1) # Instantiate the gridsearch
reg.fit(X_train_valid, y_train_valid) # Fit the grid search, i.e. perform CV and grid search. 
print("Best_pairs:", reg.best_params_ ) # The best parameter from CV
best_param.append(reg.best_params_)

print("Minimum MSE on valid set: ",metrics.mean_squared_error(reg.predict(X_valid), y_valid))

end = time.time()
print("Total time we use is: "+str(end-start))

Best_pairs: {'lasso__alpha': 10.0, 'lasso__max_iter': 90, 'lasso__tol': 0.0}
Minimum MSE on valid set:  2928751.6743318406
Total time we use is: 22.807018518447876


When we make a Lasso Regression through 5-fold cross validation model, we notice the total time has increase to around 22.81 seconds bacuse there are more calculations for this method. We want to find the best parameters with the lowest mean squared error. In this case, the alpha is 10, max_iter is 90 and tol is 0.

In [14]:
# Refit the model on the whole training set, which the selected lambda
scaler = StandardScaler().fit(X_train_valid)
X_trainS = pd.DataFrame(scaler.transform(X_train_valid))
X_testS = pd.DataFrame(scaler.transform(X_test))
X_trainS.columns = X.columns.values

# Model fitting
model1 = Lasso(alpha = 10, max_iter = 90, tol = 0)
model1.fit(X_trainS, y_train_valid)

#sort the variables
Var_coef = zip(model1.coef_ , NameOfVariables)
sorted(Var_coef)

[(-702.6363516320591, 'PctKids2Par'),
 (-363.2007820229292, 'OwnOccLowQuart'),
 (-284.29858486933784, 'PctLess9thGrade'),
 (-267.60450338766924, 'NumKidsBornNeverMar'),
 (-265.9918418234022, 'PopDens'),
 (-252.21905039413576, 'householdsize'),
 (-249.42394790166202, 'PersPerOwnOccHous'),
 (-243.35926773710744, 'PctImmigRec5'),
 (-240.6962888817962, 'PctRecentImmig'),
 (-236.92472785157082, 'PctVacMore6Mos'),
 (-229.2459934431104, 'PctRecImmig8'),
 (-221.55142080139453, 'RentLowQ'),
 (-187.61159009394342, 'MedOwnCostPctInc'),
 (-184.75920841395822, 'agePct12t29'),
 (-175.98382054702083, 'pctWRetire'),
 (-162.09283179055518, 'PctNotHSGrad'),
 (-137.0786492524251, 'PctEmplManu'),
 (-134.60368935328398, 'PctFam2Par'),
 (-125.37000068936158, 'RentHighQ'),
 (-124.08240895953317, 'PctBornSameState'),
 (-111.31137538923247, 'PctWorkMom'),
 (-109.94660064216144, 'PctBSorMore'),
 (-108.45364581214751, 'agePct16t24'),
 (-100.8192167226675, 'PctUsePubTrans'),
 (-100.45441863130395, 'PctHousOccup')

In [15]:
#coefficients shriked to 0 with 5 Fold CV
model1_coef=pd.DataFrame(zip(model1.coef_, X_train_valid.columns.values))
len(model1_coef.loc[model1_coef[0] == 0])

33

In [16]:
# Prediction evaluation
print("The prediction error on the testing set is", metrics.mean_squared_error(model1.predict(X_test), y_test))

The prediction error on the testing set is 3472450.8070895094


Then, we fit the data with the whole training set, and transform it to testing set. The result shows a prediction error on the testing set is 3472450.81, which is much better than the result we get from method 1. Totally, the coefficients of 33 variables have shrinked to zero. Therefore, among the 101 predictors, we only need to use 68 predictors, which can decrease the complexity of our model.

### Third method: 10-Fold cross validation.

In [17]:
start = time.time()

best_param=[] # Store the best hyperparameter results

# Set up the model pipeline
estimator = Pipeline(steps = [('scale', StandardScaler()), # We first need to scale the data
                     ('lasso', Lasso()) ]) # Then fit the scaled data using Lasso

parameters = {'lasso__alpha': lambdas,
             'lasso__max_iter': max_iter,
             'lasso__tol': tol}  

reg = GridSearchCV(estimator = estimator, param_grid = parameters, cv = 10, 
                   scoring = 'neg_mean_squared_error', n_jobs = -1) # Instantiate the gridsearch
reg.fit(X_train_valid, y_train_valid) # Fit the grid search, i.e. perform CV and grid search. 
print("Best_pairs:", reg.best_params_ ) # The best parameter from CV
best_param.append(reg.best_params_)

print("Minimum MSE on valid set: ",metrics.mean_squared_error(reg.predict(X_valid), y_valid))

end = time.time()
print("Total time we use is: "+str(end-start))

Best_pairs: {'lasso__alpha': 10.0, 'lasso__max_iter': 70, 'lasso__tol': 0.044444444444444446}
Minimum MSE on valid set:  2923167.118981143
Total time we use is: 45.714776039123535


When we make a Lasso Regression through 10-fold cross validation model, we notice the total time has increase to around 45.71 seconds, because 10-fold will double our calaulations. We want to find the best parameters with the lowest mean squared error. In this case, the alpha is 10, max_iter is 70 and tol is 0.044.

In [18]:
# Refit the model on the whole training set, which the selected lambda
scaler = StandardScaler().fit(X_train_valid)
X_trainS = pd.DataFrame(scaler.transform(X_train_valid))
X_testS = pd.DataFrame(scaler.transform(X_test))
X_trainS.columns = X.columns.values

# Model fitting
best_param = best_param[0] 
model2 = Lasso(alpha = best_param['lasso__alpha'], max_iter = best_param['lasso__max_iter'],
               tol = best_param['lasso__tol'])
model2.fit(X_train_valid, y_train_valid)

#sort the variables
Var_coef = zip(model2.coef_ , NameOfVariables)
sorted(Var_coef)

[(-526.1923956565603, 'PctKids2Par'),
 (-367.37047348658854, 'OwnOccLowQuart'),
 (-331.40672975616064, 'PctFam2Par'),
 (-305.7887622645784, 'agePct12t29'),
 (-304.81411242551195, 'PctLess9thGrade'),
 (-291.06160126598377, 'PctRecentImmig'),
 (-277.8154344878706, 'PopDens'),
 (-263.5235937572656, 'PersPerOwnOccHous'),
 (-259.05538180827074, 'PctImmigRec5'),
 (-258.0783556571665, 'householdsize'),
 (-244.3204854702896, 'NumKidsBornNeverMar'),
 (-234.41807627569645, 'PctVacMore6Mos'),
 (-207.57343405943195, 'RentLowQ'),
 (-187.58826246570646, 'MedOwnCostPctInc'),
 (-169.4688411683862, 'pctWRetire'),
 (-135.0886480304948, 'PctEmplManu'),
 (-132.22704689633932, 'PctBSorMore'),
 (-126.90119801493773, 'PctSpeakEnglOnly'),
 (-121.37903151448971, 'PctBornSameState'),
 (-108.48742295752014, 'PctRecImmig8'),
 (-103.78414382450681, 'PctWorkMom'),
 (-101.17607155370594, 'PctHousOccup'),
 (-96.73180329571643, 'PctUsePubTrans'),
 (-90.56055737332377, 'PctPersOwnOccup'),
 (-79.94793276401006, 'pctWPub

In [19]:
#coefficients shriked to 0 with 10 Fold CV
model2_coef=pd.DataFrame(zip(model2.coef_, X_train_valid.columns.values))
len(model2_coef.loc[model2_coef[0] == 0])

30

In [20]:
# Prediction evaluation
print("The prediction error on the testing set is", metrics.mean_squared_error(model2.predict(X_test), y_test))

The prediction error on the testing set is 3476141.483637399


Then, we fit the data with the whole training set, and transform it to testing set. The result shows a prediction error on the testing set is 3476141.48, which is good, but higher than the result of method 2. Totally, the coefficients of 30 variables have shrinked to zero. Therefore, among the 101 predictors, we only need to use 71 predictors, which can decrease the complexity of our model.

## Discussion

1. Hyperparameter selection times

The model running time for the three different Lasso regression models (train/validation/test split, 5-fold cross validation and 10-fold cross validation) are 3.85, 22.81 and 45.71 seconds. As the number of model calculations increases, the running time will increase as well. Therefor, if the users wants to save time, it's better for them to use train/validation/test split model, not the cross validation model. 

2. Coeffcients 

Among these 101 variables, we noticed that the coefficient values for these variables shrinked when the cross validation model is run. It is good for Lasso regression as it is a type of linear regression that uses shrinkage. Lasso regression wants to make data values closed to a central point, like the mean. Also, the lasso procedure encourages simple, sparse models (i.e. models with fewer parameters). 

Therefore, comparing to train/validation/test split, 5-fold cross validation and 10-fold cross validation do decrease the number of predictors, which is good for the users.

3. Prediction error

The prediction of mean squared error for the three different Lasso regression models are 3539035.59, 3472450.81 and 3476141.48, which are pretty close. 5-fold cross validation has the best prediction performance among the three models, train/validation/test split has the worst. 


In conclusion, running time is the advantage of the train/validation/test split method and prediction performance is the advantage of the cross validation method. we would suggest users to choose the 5-fold cross validation method from these three methods as it has better performance and takes less time to run. 

### Thank you